# Object Following - 物体追跡

このノートブックでは、JetBotで物体を追跡する方法を示します。 collision avoidanceをベースに、「free(直進する)」時に物体を追跡します。  
物体検出に使うモデルは一般的な90種類のオブジェクトの画像を分類した[COCOデータセット](http://cocodataset.org)を事前にトレーニングしたssd_mobilenet_v2モデルを利用します。  
このモデルはTensorRTに変換したものを使用しますが、JetPackバージョンによってTensorRTのバージョンが異なるため、変換時のTensorRTバージョンと同一の実行環境である必要があります。

追跡可能な物体はCOCOデータセットで学習している物体となります。

* 人（インデックス1）
* カップ（インデックス47）

その他多数あります（クラスインデックスの完全なリストについては、[ラベルファイル](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)で確認できます）。  
インデックス0はbackgroundになります。通常、分類・検出するモデルでは「未検出」という状態を持つためにbackgroundラベルが使われています。  
学習済みモデルは[Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)で公開されているものをベースに予めTensorRT化してあるものを使います。``Tensorflow Object Detection API``を使って自前のデータをデスクトップPCやクラウドサーバーで学習することも出来ます。

ssd_mobilenet_v2_cocoをTensorRTに変換することにより、物体検出モデルの実行が非常に高速になり、Jetson Nanoでリアルタイムに実行できるようになります。ただし、このノートブックではCOCOデータセットからのトレーニングや他の最適化に関する手順は実行しません。また、TensorRTはバージョンによりAPIが頻繁に変更されているため、他のJetPackバージョンで動作していたssd_mobilenet_v2_coco.engineは利用できません。

> このノートブックはJetson Nano 4GB JetPack 4.3で作られたJetBotで動作します。  
> Jetson Nano 2GB (JetPack 4.4以降)とJetson Nano 4GB JetPack 4.4以降では動作確認していません。

まずは始めてみましょう。

## カメラの準備
カメラを初期化しましょう。物体検出モデルは300x300ピクセルの画像を入力とするため、カメラ解像度を300x300に設定します。

> 内部的には、CameraクラスはGStreamerを使用してJetson Nanoのイメージシグナルプロセッサ（ISP）を利用しています。これはCPUでリサイズ処理を実行する場合とは比較にならないほど超高速です。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import Camera  # JetBot用に用意したカメラライブラリを利用します。

########################################
# カメラを有効化します。
# 画像はwidthとheightで指定したピクセルサイズにリサイズされます。
# ssd_mobilenet_v2_cocoは300x300の入力層のため、カメラ画像は300x300にリサイズします。
# fpsのデフォルトは21ですが、カメラフレーム更新に連動して推論を実行するようにコーディングしているため、
# 処理が重くなってしまいます。そのためfpsを小さく設定します。
########################################
camera = Camera(width=300, height=300, fps=5)

事前トレーニング済みのSSDエンジンを使用する[ObjectDetector](https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/jetbot/object_detection.py)クラスをインポートして、``ssd_mobilenet_v2_coco.engine``をロードします。

Jetpack4.3向けの[ssd_monbilenet_v2_coco.engine](https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view) をダウンロードし、JupyterLabの本Notebookと同じフォルダに``ssd_monbilenet_v2_coco.engine``をアップロードします。

### SSD MobileNet V2モデルを読み込む

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import ObjectDetector  # JetBot用に用意した物体検出ライブラリを利用します。

########################################
# TensorRTの物体検出モデルを読み込みます。
########################################
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

内部的には、``ObjectDetector``クラスはTensorRT Python APIを使用してモデルを実行します。また、モデルへの入力の前処理や、検出されたオブジェクトの解析も行います。 現時点では、``jetbot.ssd_tensorrt``パッケージを使用して作成されたモデルでのみ機能します。このパッケージには、モデルをTensorflowオブジェクト検出APIから最適化されたTensorRTエンジンに変換するためのユーティリティが含まれています。

次に、カメラ入力を使用してネットワークを実行してみましょう。 デフォルトでは ``ObjectDetector``クラスはカメラが生成する``bgr8``フォーマットを期待しています。 しかし、別のフォーマットを入力に使う場合は、デフォルトの前処理関数をオーバーライドして変更できます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import cv2

########################################
# 物体検出はTensorflowで学習されたモデルを使っています。このモデルは学習時にRGB画像フォーマットで学習されています。
# そのモデルをTensorRTモデルに変換したものがssd_mobilenet_v2_coco.engineです。
# 物体検出モデルの入力データはRGBフォーマットに変換する方が精度がよくなりますが、
# BGR->RGB変換をObjectDetectorクラスが実行しているため、
# このノートブックにおける物体検出の入力データはOpenCVカメラ画像のBGRフォーマットのまま渡すことになります。
########################################
detections = model(camera.value)

print(detections)  # 推論結果を表示します。

カメラ画像にCOCOオブジェクトがある場合、その情報は``detections``変数に格納されています。

### テキスト領域に検出を表示する

次のコードを使用して、検出されたオブジェクトの情報をテキストエリアに表示します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()  # テキストウィジェットを作成します。

detections_widget.value = str(detections)  # テキストウィジェットに検出したオブジェクトの情報を反映します。

display(detections_widget)  # テキストウィジェットを表示します。

カメラ画像で検出された各オブジェクトのラベルID、信頼度、境界ボックスの座標が表示されます。

ミニバッチ学習時に複数の画像を一度に学習したなごりで、予測時にも一度に複数の画像を入力として期待するモデルに仕上がっています。  
今回は1台のカメラしか使わないため、モデルの入力には1枚の画像を持つ配列が使われています。  
最初の画像で検出された最初のオブジェクトのみを表示するには、次のように呼び出すことができます。

> オブジェクトが検出されない場合、エラーになるため、try-exceptでエラーハンドリングします

In [ ]:
image_number = 0  # 推論時に配列で与えた最初の画像を表す配列のインデックス番号。推論時は1枚の画像しか与えていないため、0固定値。
object_number = 0  # 検出した物体の情報を持つ配列から、取り出したい配列のインデックス番号。複数の物体が検出された場合は0以外もあり得る。検出しなかった場合、配列は存在しない。

try:
    print(detections[image_number][object_number])
except:
    print("object not found")

### 中心物体を追跡するようにロボットを制御する

次に、ロボットに指定されたクラスのオブジェクトを追跡させます。 これを行うには、次のようにします

1.  指定したクラスに一致するオブジェクトを検出します。[ラベルファイル](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)でラベルIDと対応する物体を確認してください。
2.  カメラの視野の中心に最も近いオブジェクトを選択します。これが指定したオブジェクトの時に追跡するターゲットになります。
3.  ロボットをターゲットオブジェクトに向けます。
4.  collision avoidanceをベース動作にしているため、障害物によってブロックされていると判断した場合は、左折します。

> ラベルファイルにはいくつかバージョンがあります。Tensorflowのラベルは80オブジェクト分になります。  
そのため、いくつか名前のないラベルが含まれています。[cocoデータセットのラベルについて](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/)

また、ターゲットオブジェクトのラベル、ロボットの速度を制御するために使用するいくつかのウィジェットを作成します。
`turn gain`は、ターゲットオブジェクトとロボットの視野の中心との間の距離に基づいてロボットが回転する速度を制御します。

まず、衝突回避モデルをロードします。
衝突回避の例に従って、実際の環境でうまく動作するモデルを使用することをお勧めします。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

########################################
# 衝突回避モデルを読み込みます。
########################################
collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))

########################################
# GPU処理が可能な部分をGPUで処理するように設定します。
# モデルを評価モードにします。
# モデルをfloat16型に変換します。
########################################
device = torch.device('cuda')
collision_model = collision_model.to(device)
collision_model = collision_model.eval().half()

########################################
# この値はpytorch ImageNetの学習に使われた正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリングすること）のパラメータです。
# カメラ画像はこの値でRGBを正規化することが望ましいでしょう。
# ここではtransforms.ToTensor()を使っていないため、正規化前のRGB値の範囲は[0, 255]です。
# そこで、学習時のRGB各範囲と同じ範囲にスケーリングするように正規化パラメータに255.0を掛けて設定します。
########################################
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

########################################
# 正規化する関数を定義します。
# torchvision.transforms.Normalizeクラスはインスタンス化すると
# torch.nn.functional.normalize関数を返します。
# ソースコード：
#   https://pytorch.org/docs/stable/_modules/torchvision/transforms/transforms.html#Normalize
#   https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#normalize
#########################################
normalize = torchvision.transforms.Normalize(mean, stdev)

########################################
# カメラ画像をモデル入力用データに変換します。
########################################
def preprocess(camera_value):
    # OpenCVで取得したカメラ画像を変数xにコピーします。
    x = camera_value
    # 画像解像度を300x300から224x224に変更します。
    x = cv2.resize(x, (224, 224))
    # 学習時の画像データはtorchvision.datasets.ImageFolderを使って読み込んでいるため、モデルはRGBフォーマットの画像で学習しています。
    # カメラ映像はOpenCVで読み込んでいるため画像はBGRフォーマットになっています。これをRGBフォーマットに変換します。    
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    # 画像フォーマットをHWCからCHWに変換します。
    x = x.transpose((2, 0, 1))
    # float32に変換します。
    x = torch.from_numpy(x).float()
    # 正規化します。
    x = normalize(x)
    # GPUデバイスを利用します。float16に変換します。
    x = x.to(device).half()
    # バッチ配列に変換します。
    x = x[None, ...]
    # 入力用データxを返します。
    return x

モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import Robot  # JetBotを制御するためのライブラリを利用します。

########################################
# JetBotの制御用クラスをインスタンス化します。
########################################
robot = Robot()

コントロールウィジェットとカメラ更新とモデル実行の関数を作成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import bgr8_to_jpeg  # JetBot用に用意した画像変換ライブラリを利用します。

########################################
# 「blocked」の確率を表示するためのスライダーを用意します。
########################################
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')

########################################
# 画像表示用のウィジェットを用意します。
# widthとheightは表示するウィジェットの幅と高さです。
# カメラ画像サイズと一致する必要はありません。
########################################
image_widget = widgets.Image(format='jpeg', width=300, height=300)

########################################
# 追跡対象のラベル名を選択するためのウィジェットを作成します。
# ラベル名は学習済みモデルssd_mobilenet_v2_coco.engineが持つラベル名になります。
# 追跡対象はpersonとしておきます。
########################################
label_widget = widgets.Dropdown(
    options=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
             'fire hydrant', '12', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
             'cow', 'elephant', 'bear', 'zebra', 'giraffe', '26', 'backpack', 'umbrella', '29', '30',
             'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
             'skateboard', 'surfboard', 'tennis racket', 'bottle', '45', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
             'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
             'cake', 'chair', 'couch', 'potted plant', 'bed', '66', 'dining table', '68', '69', 'toilet',
             '71', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
             'sink', 'refrigerator', '83', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'],
    value='person',
    description='tracked label',
    disabled=False
)

########################################
# JetBotの動作を調整するためのスライダーウィジェットを用意します。
########################################
speed_widget = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

########################################
# ウィジェットの画像サイズを取得しておきます。
########################################
width = int(image_widget.width)
height = int(image_widget.height)

########################################
# 描画する文字のサイズを自動調整します。
########################################
fontScale = height/1000.0
if fontScale < 0.4:
    fontScale = 0.4
fontThickness = 1 + int(fontScale)
fontFace = cv2.FONT_HERSHEY_SIMPLEX

########################################
# カメラ画像の中央を原点とした、
# 検出した追跡対象の中央座標(center_x, center_y)を取得します。
########################################
def detection_center(detection):
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
########################################
# カメラ画像の中央と追跡対象の中央までの距離を取得します。
########################################
def norm(vec):
    return np.sqrt(vec[0]**2 + vec[1]**2)

########################################
# 複数の追跡対象ターゲットのうち、もっとも画面中央に映っているターゲットを取得します。
########################################
def closest_detection(detections):
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

########################################
# カメラ画像が更新されたときに実行する処理を定義します。
########################################
def execute(change):
    # カメラ画像を変数imageにコピーします。
    image = change['new']
        
    ####################
    # 衝突回避モデルを実行して、「blocked」かどうかを判断します。
    ####################
    # 推論を実行します。
    collision_output = collision_model(preprocess(image))
    # collision_output.flatten()を呼び出すことで可能な限り不要な次元を除去します。([[blocked_rate, free_rate]]を[blocked_rate, free_rate]に変換)
    # softmax()関数を適用して出力ベクトルの合計が1になるように正規化します（これにより確率分布になります）
    # 入力データは多次元のバッチ配列になっています。出力もそれに対応しているためcollision_output.flatten()は多次元配列になっています。
    # そのうえで、「blocked」の確率となるcollision_output.flatten()[0]の値を取得します。「free」の確率を取得する場合はcollision_output.flatten()[1]になります。
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    # 「blocked」の確率をスライダーに反映します。
    blocked_widget.value = prob_blocked
    
    ####################
    # 「blocked」の確率が50%未満なら直進します。
    # 画像表示ウィジェットを更新します。
    # この関数の処理をここで終了します。
    ####################
    if prob_blocked > 0.5:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
    
    ####################
    # 「blocked」の確率が50%以下なら、つまり「free」なら物体検出を実行します。
    ####################
    # 物体検出モデルの実行コード内でBGR->RGB変換をおこなっているため、
    # ここではOpenCVカメラ画像のBGRフォーマットのまま渡します。
    detections = model(image)
    
    # 検出した物体の情報を表示します。
    display_str = []
    display_str.append("detection info")
    for det in detections[0]:  # 検出した物体を一つ一つ解析します。
        if det['label']  == 0:  # 検出結果のうち、ラベル番号0は背景のためスキップします。
            # background. skip
            continue
        if det['confidence'] <= 0.2:  # スコアが低い場合。ここでは確認のために検出したものとしてpassします。
            # bad score. skip
            #continue
            pass
        bbox = det['bbox']  # 検出した物体の範囲を表す長方形のx,y座標を取得します。
        score = det['confidence']  # 検出した物体のスコア（確率）を取得します。
        label = det['label']  # 検出した物体のラベル番号を取得します。
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)  # 検出した物体を青色の長方形で囲みます。
        display_str.append("label:{} score:{:.2f}".format(label_widget.options[int(label)-1], score))  # ラベル名とスコアを文字列の配列に追加します。
        #cv2.putText(image, display_str, org=(10, 20+20*num_detection), fontFace=fontFace, fontScale=fontScale, thickness=fontThickness, color=(77, 255, 9))

    ####################
    # 検出した物体のラベル名とスコアを画像に描画します。
    # 描画位置やパディングは画像サイズと文字数から、見やすくなるように計算して描画します。
    ####################
    max_text_width = 0
    max_text_height = 0
    if len(display_str) > 0:
        [(text_width, text_height), baseLine] = cv2.getTextSize(text=display_str[0], fontFace=fontFace, fontScale=fontScale, thickness=fontThickness)
        x_left = int(baseLine)
        y_top = int(baseLine)
        for i in range(len(display_str)):
            [(text_width, text_height), baseLine] = cv2.getTextSize(text=display_str[i], fontFace=fontFace, fontScale=fontScale, thickness=fontThickness)
            if max_text_width < text_width:
                max_text_width = text_width
            if max_text_height < text_height:
                max_text_height = text_height
        for i in range(len(display_str)):
            cv2.putText(image, display_str[i], org=(x_left, y_top + int(max_text_height*1.2 + (max_text_height*1.2 * i))), fontFace=fontFace, fontScale=fontScale, thickness=fontThickness, color=(77, 255, 9))

    ####################
    # 検出した物体が追跡対象のラベルと一致している場合、その情報を取得します。
    # ラベル番号は、物体検出の結果は0が背景、1が「person」ラベルになります。
    # ラベル選択ウィジェットのドロップダウンリストの配列は背景を選択しないようにしているため、
    # 0が「person」ラベルになります。そのため、ラベル選択ウィジェットのインデックスに+1したものが物体検出のラベル番号と一致することになります。
    ####################
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.index)+1]
    
    ####################
    # 追跡対象の物体のうち、画面中央にもっとも近い物体を追跡対象とします。
    ####################
    target = closest_detection(matching_detections)
    
    ####################
    # 追跡対象となる物体が存在する場合は、物体を緑色の長方形で囲みます。
    ####################
    if target is not None:
        bbox = target['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)

    ####################
    # 追跡対象となる物体が存在しない場合は、衝突回避の「free」と同じように前進します。
    ####################
    if target is None:
        robot.forward(float(speed_widget.value))
        
    ####################
    # 追跡対象となる物体が存在する場合は、物体の中心方向を向くようにモーターを制御します。
    ####################
    else:
        center = detection_center(target)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    ####################
    # 画像表示ウィジェットを更新します。
    ####################
    image_widget.value = bgr8_to_jpeg(image)

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラ画像の更新に連動して動作させる必要があります。

JetBotでは、traitlets.HasTraitsを継承したCameraクラスを実装しているので、observe()を呼び出すだけで実現できます。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  
``start jetbot``ボタンを押すとモデルの初期化が実行され、JetBotが動作し始めます。  
``stop jetbot``ボタンを押すとJetBotが停止します。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import ipywidgets
import time

########################################
# スタートボタンとストップボタンを作成します。
########################################
model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

########################################
# スタートボタンがクリックされた時に呼び出す関数を定義します。
########################################
def start_model(c):
    execute({'new': camera.value})  # execute()関数を1回呼び出して初期化します。
    camera.observe(execute, names='value')  # Cameraクラスのtraitlets.Any()型のvalue変数(カメラ画像データ)が更新されたときに指定した関数を呼び出します。
model_start_button.on_click(start_model)  # startボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ストップボタンがクリックされた時に呼び出す関数を定義します。
########################################
def stop_model(c):
    camera.unobserve(execute, names='value')  # カメラ画像データの更新と指定した関数の連動を解除します。
    time.sleep(1)  # 実行中の処理が完了するまで少し待ちます。
    robot.stop()  # モーターを停止します。
model_stop_button.on_click(stop_model)  # stopボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ウィジェットの表示レイアウトを定義します。
########################################
model_widget = ipywidgets.VBox([
    image_widget,
    ipywidgets.HBox([label_widget, blocked_widget]),
    ipywidgets.HBox([speed_widget, turn_gain_widget]),
    ipywidgets.HBox([model_start_button, model_stop_button])
])

########################################
# ウィジェットを表示します。
########################################
display(model_widget)

うごいた！  
ターゲットが検出されると緑色のボックスが表示され、ターゲット以外の検出された物体は青色のボックスで表示されます。  
衝突回避モデルによって「blocked(旋回する)」と判断された時、JetBotは左に曲がります。  
衝突回避モデルによって「free(直進する)」と判断された時、ターゲットを検出している場合はJetBotはターゲットを追跡するように動作します。  
衝突回避モデルによって「free(直進する)」と判断された時、ターゲットを検出していない場合は衝突回避モデルと同様に直進します。

# カメラの停止
最後に、他のノートブックでカメラを使うために、このノートブックで使ったカメラを停止しておきます。

In [ ]:
camera.stop()  # カメラを停止します。